# MLOps Lab 2 Assignment (MLFlow)

Author: Grant Nitta

Date Created: 03/20/2025

Date Last Modified: 03/21/2025

# Task

Once you have selected a set of data, create a brand new experiment in MLFlow and begin exploring your data. Do some EDA, clean up, and learn about your data. You do not need to begin tracking anything yet, but you can if you want to (e.g. you can log different versions of your data as you clean it up and do any feature engineering). Do not spend a ton of time on this part. Your goal isn't really to build a great model, so don't spend hours on feature engineering and missing data imputation and things like that.

Once your data is clean, begin training models and tracking your experiments. If you intend to use this same dataset for your final project, then start thinking about what your model might look like when you actually deploy it. For example, when you engineer new features, be sure to save the code that does this, as you will need this in the future. If your final model has 1000 complex features, you might have a difficult time deploying it later on. If your final model takes 15 minutes to train, or takes a long time to score a new batch of data, you may want to think about training a less complex model.

Now, when tracking your experiments, at a *minimum*, you should:

1. Try at least 3 different ML algorithms (e.g. linear regression, decision tree, random forest, etc.).
2. Do hyperparameter tuning for **each** algorithm.
3. Do some very basic feature selection, and repeat the above steps with these reduced sets of features.
4. Identify the top 3 best models and note these down for later.
6. Choose the **final** "best" model that you would deploy or use on future data, stage it (in MLFlow), and run it on the test set to get a final measure of performance. Don't forget to log the test set metric.
7. Be sure you logged the exact training, validation, and testing datasets for the 3 best models, as well as hyperparameter values, and the values of your metrics.  
8. Push your code to Github. No need to track the mlruns folder, the images folder, any datasets, or the sqlite database in git.

### Turning It In

In the MLFlow UI, next to the refresh button you should see three vertical dots. Click the dots and then download your experiments as a csv file. Open the csv file in Excel and highlight the rows for your top 3 models from step 4, highlight the run where you applied your best model to the test set, and then save as an excel file. Take a snapshot of the Models page in the MLFLow UI showing the model you staged in step 6 above. Submit the excel file and the snapshot to Canvas.

# Library Importation

In [1]:
import mlflow
import pandas as pd
import numpy as np

from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import cross_val_score, train_test_split

from statsmodels.stats.outliers_influence import variance_inflation_factor


from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

# Data Importation

In [2]:
# pip install ucimlrepo
# !pip install statsmodels

In [3]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
student_performance = fetch_ucirepo(id=320)

# data (as pandas dataframes)
X = student_performance.data.features
y = student_performance.data.targets

# metadata
# print(student_performance.metadata)

# # variable information
# print(student_performance.variables)

# Setting up MLFlow

In [4]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("Lab2-student_performance_V5")

2025/03/21 20:14:08 INFO mlflow.tracking.fluent: Experiment with name 'Lab2-student_performance_V5' does not exist. Creating a new experiment.


<Experiment: artifact_location='/Users/skier/MSDS/Spring2/Spring2-MSDS-MLOps/labs/lab2/mlruns/9', creation_time=1742613248730, experiment_id='9', last_update_time=1742613248730, lifecycle_stage='active', name='Lab2-student_performance_V5', tags={}>

# Experimenting Phase 1

In [5]:
X_encoded = X.copy()

# Track column transformations
column_mapping = {}
label_encoders = {}

# Find all object and category columns
string_columns = X.select_dtypes(include=["object", "category"]).columns

for col in string_columns:
    # For columns with many unique values, use label encoding
    le = LabelEncoder()
    X_encoded[col + "_encoded"] = le.fit_transform(X[col])

    # Drop the original column
    X_encoded = X_encoded.drop(col, axis=1)

    # Store mapping information
    column_mapping[col] = [col + "_encoded"]
    label_encoders[col] = le

X_encoded = X_encoded.astype(float)

In [6]:
y_use = y["G1"]

In [7]:
X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y_use, test_size=0.2, shuffle=True
)
X_train_val, X_val, y_train_val, y_val = train_test_split(
    X_train, y_train, test_size=0.2, shuffle=True
)

# Training

In [8]:
def objective(params):
    with mlflow.start_run():
        classifier_type = params["type"]
        del params["type"]
        if classifier_type == "dt":
            clf = DecisionTreeClassifier(**params)
        elif classifier_type == "rf":
            clf = RandomForestClassifier(**params)
        elif classifier_type == "gb":
            clf = GradientBoostingClassifier(**params)
        else:
            return 0
        acc = cross_val_score(clf, X_train_val, y_train_val).mean()

        mlflow.set_tag("Model", classifier_type)

        mlflow.log_params(params)
        mlflow.log_metric("accuracy", acc)
        mlflow.end_run()
        return {"loss": -acc, "status": STATUS_OK}


search_space = hp.choice(
    "classifier_type",
    [
        {
            "type": "dt",
            "criterion": hp.choice("dtree_criterion", ["gini", "entropy"]),
            "max_depth": hp.choice(
                "dtree_max_depth",
                [None, hp.randint("dtree_max_depth_int", 1, 10)],
            ),
            "min_samples_split": hp.randint("dtree_min_samples_split", 2, 10),
        },
        {
            "type": "rf",
            "n_estimators": hp.randint("rf_n_estimators", 20, 500),
            "max_features": hp.randint("rf_max_features", 2, 9),
            "criterion": hp.choice("criterion", ["gini", "entropy"]),
        },
        {
            "type": "gb",
            "loss": hp.choice("gb_loss", ["log_loss"]),
            "learning_rate": hp.uniform("gb_learning_rate", 0.05, 2),
            "n_estimators": hp.randint("gb_n_estimators", 20, 500),
            "subsample": hp.uniform("gb_subsample", 0.1, 1),
            "criterion": hp.choice(
                "gb_criterion", ["friedman_mse", "squared_error"]
            ),
            "max_depth": hp.choice(
                "gb_max_depth",
                [None, hp.randint("gb_max_depth_int", 1, 10)],
            ),
        },
    ],
)

algo = tpe.suggest
trials = Trials()

In [9]:
best_result = fmin(
    fn=objective, space=search_space, algo=algo, max_evals=32, trials=trials
)

  0%|          | 0/32 [00:00<?, ?trial/s, best loss=?]

  3%|▎         | 1/32 [00:00<00:03,  9.58trial/s, best loss: -0.15180722891566265]

/opt/anaconda3/envs/mlops/lib/python3.12/site-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(

/opt/anaconda3/envs/mlops/lib/python3.12/site-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(

/opt/anaconda3/envs/mlops/lib/python3.12/site-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(



  9%|▉         | 3/32 [00:12<02:15,  4.67s/trial, best loss: -0.15180722891566265]

/opt/anaconda3/envs/mlops/lib/python3.12/site-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(



...

 94%|█████████▍| 30/32 [01:09<00:01,  1.39trial/s, best loss: -0.18795180722891563]

/opt/anaconda3/envs/mlops/lib/python3.12/site-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(



 97%|█████████▋| 31/32 [01:10<00:00,  1.74trial/s, best loss: -0.18795180722891563]

/opt/anaconda3/envs/mlops/lib/python3.12/site-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(



100%|██████████| 32/32 [01:11<00:00,  2.23s/trial, best loss: -0.18795180722891563]


In [10]:
best_result

{'classifier_type': 1,
 'criterion': 0,
 'rf_max_features': 2,
 'rf_n_estimators': 435}

# Feature Selection

In [11]:
X_redcued_train = X_train.copy()
VIF = [0]
while len(VIF) > 0:
    X_numeric = pd.DataFrame()
    for col in X_redcued_train.columns:
        # Force everything through string conversion to be safe
        X_numeric[col] = pd.to_numeric(
            X_train[col].astype(str), errors="coerce"
        )

    # Now calculate VIF
    vif_data = pd.DataFrame()
    vif_data["Feature"] = X_numeric.columns
    vif_data["VIF"] = [
        variance_inflation_factor(X_numeric.values, i)
        for i in range(X_numeric.shape[1])
    ]
    vif_test = vif_data.set_index("Feature").sort_values(
        by="VIF", ascending=False
    )
    if vif_test.max().iloc[0] > 5:
        print(VIF)
        VIF = vif_test.idxmax().iloc[0]
        X_redcued_train.drop(VIF, axis=1, inplace=True)
    else:
        print("Stopping")
        VIF = []

[0]
age
famrel
Medu
freetime
higher_encoded
goout
Pstatus_encoded
Fjob_encoded
Walc
health
studytime
Stopping


In [12]:
def objective(params):
    with mlflow.start_run():
        classifier_type = params["type"]
        del params["type"]
        if classifier_type == "dt":
            clf = DecisionTreeClassifier(**params)
        elif classifier_type == "rf":
            clf = RandomForestClassifier(**params)
        elif classifier_type == "gb":
            clf = GradientBoostingClassifier(**params)
        else:
            return 0
        acc = cross_val_score(clf, X_redcued_train, y_train).mean()

        mlflow.set_tag("Model", classifier_type)
        mlflow.set_tag("Data", "Training")
        mlflow.log_params(params)
        mlflow.log_metric("accuracy", acc)
        mlflow.sklearn.log_model(clf, artifact_path="better_models")
        mlflow.end_run()
        return {"loss": -acc, "status": STATUS_OK}


search_space = hp.choice(
    "classifier_type",
    [
        {
            "type": "dt",
            "criterion": hp.choice("dtree_criterion", ["gini", "entropy"]),
            "max_depth": hp.choice(
                "dtree_max_depth",
                [None, hp.randint("dtree_max_depth_int", 1, 10)],
            ),
            "min_samples_split": hp.randint("dtree_min_samples_split", 2, 10),
        },
        {
            "type": "rf",
            "n_estimators": hp.randint("rf_n_estimators", 20, 500),
            "max_features": hp.randint("rf_max_features", 2, 9),
            "criterion": hp.choice("criterion", ["gini", "entropy"]),
        },
        {
            "type": "gb",
            "loss": hp.choice("gb_loss", ["log_loss"]),
            "learning_rate": hp.uniform("gb_learning_rate", 0.05, 2),
            "n_estimators": hp.randint("gb_n_estimators", 20, 500),
            "subsample": hp.uniform("gb_subsample", 0.1, 1),
            "criterion": hp.choice(
                "gb_criterion", ["friedman_mse", "squared_error"]
            ),
            "max_depth": hp.choice(
                "gb_max_depth",
                [None, hp.randint("gb_max_depth_int", 1, 10)],
            ),
        },
    ],
)

algo = tpe.suggest
trials = Trials()

In [13]:
best_result = fmin(
    fn=objective, space=search_space, algo=algo, max_evals=32, trials=trials
)

  0%|          | 0/32 [00:00<?, ?trial/s, best loss=?]

/opt/anaconda3/envs/mlops/lib/python3.12/site-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(

2025/03/21 20:15:39 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.



  3%|▎         | 1/32 [00:17<09:16, 17.95s/trial, best loss: -0.10797236743838685]

/opt/anaconda3/envs/mlops/lib/python3.12/site-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(

2025/03/21 20:15:42 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.



  6%|▋         | 2/32 [00:21<04:39,  9.31s/trial, best loss: -0.12339432412247946]

...

 94%|█████████▍| 30/32 [02:04<00:03,  1.55s/trial, best loss: -0.16376026885735623]

/opt/anaconda3/envs/mlops/lib/python3.12/site-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(

2025/03/21 20:17:26 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.



 97%|█████████▋| 31/32 [02:05<00:01,  1.34s/trial, best loss: -0.1676250933532487] 

/opt/anaconda3/envs/mlops/lib/python3.12/site-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(

2025/03/21 20:17:26 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.



100%|██████████| 32/32 [02:05<00:00,  3.93s/trial, best loss: -0.1676250933532487]


In [14]:
best_result

{'classifier_type': 0,
 'dtree_criterion': 0,
 'dtree_max_depth': 1,
 'dtree_max_depth_int': 1,
 'dtree_min_samples_split': 2}

# Top 3 Best models Training

1. bb5a5bb1870749aa9dfe9e49bed03193
2. 235d2f5de03f4a9fb8b16e724cf96418
3. 6a82bcec3a1a48d6aa0196d67887e8f9

# Running On Validation

In [15]:
X_redcued_val = X_val.copy()
VIF = [0]
while len(VIF) > 0:
    X_numeric = pd.DataFrame()
    for col in X_redcued_val.columns:
        # Force everything through string conversion to be safe
        X_numeric[col] = pd.to_numeric(
            X_train[col].astype(str), errors="coerce"
        )

    # Now calculate VIF
    vif_data = pd.DataFrame()
    vif_data["Feature"] = X_numeric.columns
    vif_data["VIF"] = [
        variance_inflation_factor(X_numeric.values, i)
        for i in range(X_numeric.shape[1])
    ]
    vif_test = vif_data.set_index("Feature").sort_values(
        by="VIF", ascending=False
    )
    if vif_test.max().iloc[0] > 5:
        print(VIF)
        VIF = vif_test.idxmax().iloc[0]
        X_redcued_val.drop(VIF, axis=1, inplace=True)
    else:
        print("Stopping")
        VIF = []

[0]
age
famrel
Medu
freetime
higher_encoded
goout
Pstatus_encoded
Fjob_encoded
Walc
health
studytime
Stopping


In [16]:
def objective(params):
    with mlflow.start_run():
        classifier_type = params["type"]
        del params["type"]
        if classifier_type == "dt":
            clf = DecisionTreeClassifier(**params)
        elif classifier_type == "rf":
            clf = RandomForestClassifier(**params)
        elif classifier_type == "gb":
            clf = GradientBoostingClassifier(**params)
        else:
            return 0
        acc = cross_val_score(clf, X_redcued_val, y_val).mean()

        mlflow.set_tag("Model", classifier_type)
        mlflow.set_tag("Data", "Validation")
        mlflow.log_params(params)
        mlflow.log_metric("accuracy", acc)
        mlflow.sklearn.log_model(clf, artifact_path="better_models")
        mlflow.end_run()
        return {"loss": -acc, "status": STATUS_OK}


search_space = hp.choice(
    "classifier_type",
    [
        {
            "type": "dt",
            "criterion": hp.choice("dtree_criterion", ["gini", "entropy"]),
            "max_depth": hp.choice(
                "dtree_max_depth",
                [None, hp.randint("dtree_max_depth_int", 1, 10)],
            ),
            "min_samples_split": hp.randint("dtree_min_samples_split", 2, 10),
        },
        {
            "type": "rf",
            "n_estimators": hp.randint("rf_n_estimators", 20, 500),
            "max_features": hp.randint("rf_max_features", 2, 9),
            "criterion": hp.choice("criterion", ["gini", "entropy"]),
        },
        {
            "type": "gb",
            "loss": hp.choice("gb_loss", ["log_loss"]),
            "learning_rate": hp.uniform("gb_learning_rate", 0.05, 2),
            "n_estimators": hp.randint("gb_n_estimators", 20, 500),
            "subsample": hp.uniform("gb_subsample", 0.1, 1),
            "criterion": hp.choice(
                "gb_criterion", ["friedman_mse", "squared_error"]
            ),
            "max_depth": hp.choice(
                "gb_max_depth",
                [None, hp.randint("gb_max_depth_int", 1, 10)],
            ),
        },
    ],
)

algo = tpe.suggest
trials = Trials()

In [17]:
best_result_val = fmin(
    fn=objective, space=search_space, algo=algo, max_evals=32, trials=trials
)
best_result_val

  0%|          | 0/32 [00:00<?, ?trial/s, best loss=?]

/opt/anaconda3/envs/mlops/lib/python3.12/site-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(

2025/03/21 20:17:31 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.



  3%|▎         | 1/32 [00:04<02:04,  4.02s/trial, best loss: -0.13476190476190475]

/opt/anaconda3/envs/mlops/lib/python3.12/site-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(

2025/03/21 20:17:32 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.



  6%|▋         | 2/32 [00:04<01:05,  2.18s/trial, best loss: -0.1433333333333333] 

...

/opt/anaconda3/envs/mlops/lib/python3.12/site-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(

2025/03/21 20:18:17 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.



 97%|█████████▋| 31/32 [00:50<00:02,  2.14s/trial, best loss: -0.25]

/opt/anaconda3/envs/mlops/lib/python3.12/site-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(

2025/03/21 20:18:18 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.



100%|██████████| 32/32 [00:50<00:00,  1.59s/trial, best loss: -0.27]


{'classifier_type': 0,
 'dtree_criterion': 1,
 'dtree_max_depth': 1,
 'dtree_max_depth_int': 3,
 'dtree_min_samples_split': 4}

# Top 3 Validation

1. a4c59bd8893f4bb6b71695abb578a662
2. 9c2f5d5f440947e8b485214aafe5793e
3. 0d5a121206cd4dda9fc06a73104b1fd2

# Running on Test Data

In [18]:
X_redcued_test = X_test.copy()
VIF = [0]
while len(VIF) > 0:
    X_numeric = pd.DataFrame()
    for col in X_redcued_test.columns:
        # Force everything through string conversion to be safe
        X_numeric[col] = pd.to_numeric(
            X_train[col].astype(str), errors="coerce"
        )

    # Now calculate VIF
    vif_data = pd.DataFrame()
    vif_data["Feature"] = X_numeric.columns
    vif_data["VIF"] = [
        variance_inflation_factor(X_numeric.values, i)
        for i in range(X_numeric.shape[1])
    ]
    vif_test = vif_data.set_index("Feature").sort_values(
        by="VIF", ascending=False
    )
    if vif_test.max().iloc[0] > 5:
        # print(VIF)
        VIF = vif_test.idxmax().iloc[0]
        X_redcued_test.drop(VIF, axis=1, inplace=True)
    else:
        # print("Stopping")
        VIF = []

In [19]:
def objective(params):
    with mlflow.start_run():
        classifier_type = params["type"]
        del params["type"]
        if classifier_type == "dt":
            clf = DecisionTreeClassifier(**params)
        elif classifier_type == "rf":
            clf = RandomForestClassifier(**params)
        elif classifier_type == "gb":
            clf = GradientBoostingClassifier(**params)
        else:
            return 0
        acc = cross_val_score(clf, X_redcued_test, y_test).mean()

        mlflow.set_tag("Model", classifier_type)
        mlflow.set_tag("Data", "Test")
        mlflow.log_params(params)
        mlflow.log_metric("accuracy", acc)
        mlflow.sklearn.log_model(clf, artifact_path="better_models")
        mlflow.end_run()
        return {"loss": -acc, "status": STATUS_OK}


search_space = hp.choice(
    "classifier_type",
    [
        {
            "type": "dt",
            "criterion": hp.choice("dtree_criterion", ["gini", "entropy"]),
            "max_depth": hp.choice(
                "dtree_max_depth",
                [None, hp.randint("dtree_max_depth_int", 1, 10)],
            ),
            "min_samples_split": hp.randint("dtree_min_samples_split", 2, 10),
        },
        {
            "type": "rf",
            "n_estimators": hp.randint("rf_n_estimators", 20, 500),
            "max_features": hp.randint("rf_max_features", 2, 9),
            "criterion": hp.choice("criterion", ["gini", "entropy"]),
        },
        {
            "type": "gb",
            "loss": hp.choice("gb_loss", ["log_loss"]),
            "learning_rate": hp.uniform("gb_learning_rate", 0.05, 2),
            "n_estimators": hp.randint("gb_n_estimators", 20, 500),
            "subsample": hp.uniform("gb_subsample", 0.1, 1),
            "criterion": hp.choice(
                "gb_criterion", ["friedman_mse", "squared_error"]
            ),
            "max_depth": hp.choice(
                "gb_max_depth",
                [None, hp.randint("gb_max_depth_int", 1, 10)],
            ),
        },
    ],
)

algo = tpe.suggest
trials = Trials()

In [20]:
best_result_test = fmin(
    fn=objective, space=search_space, algo=algo, max_evals=32, trials=trials
)

  0%|          | 0/32 [00:00<?, ?trial/s, best loss=?]

/opt/anaconda3/envs/mlops/lib/python3.12/site-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(

2025/03/21 20:18:20 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.



  3%|▎         | 1/32 [00:01<00:48,  1.57s/trial, best loss: -0.07692307692307693]

/opt/anaconda3/envs/mlops/lib/python3.12/site-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(

2025/03/21 20:18:21 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.



  6%|▋         | 2/32 [00:02<00:33,  1.12s/trial, best loss: -0.14615384615384613]

...

 94%|█████████▍| 30/32 [00:44<00:01,  1.20trial/s, best loss: -0.1769230769230769]

/opt/anaconda3/envs/mlops/lib/python3.12/site-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(

2025/03/21 20:19:05 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.



 97%|█████████▋| 31/32 [00:45<00:00,  1.21trial/s, best loss: -0.1769230769230769]

/opt/anaconda3/envs/mlops/lib/python3.12/site-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(

2025/03/21 20:19:05 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.



100%|██████████| 32/32 [00:46<00:00,  1.46s/trial, best loss: -0.1769230769230769]


In [21]:
best_result_test

{'classifier_type': 0,
 'dtree_criterion': 1,
 'dtree_max_depth': 1,
 'dtree_max_depth_int': 5,
 'dtree_min_samples_split': 6}

# Top 3 models from training using Validation and Test data

In [22]:
# mlflow.set_experiment("Lab2-student_performance_output")

In [30]:
top3_1_logged_model = "runs:/cc65639d79bd4d4fa2c85e4a23129326/better_models"  # replace with one of your models

# Load model as a PyFuncModel.
top3_1_loaded_model = mlflow.pyfunc.load_model(top3_1_logged_model)

top3_2_logged_model = "runs:/241a552635ef442b85a827ec1f17d58b/better_models"  # replace with one of your models

# Load model as a PyFuncModel.
top3_2_loaded_model = mlflow.pyfunc.load_model(top3_2_logged_model)

top3_3_logged_model = "runs:/5c804a1d1641450496d327a61bb590ff/better_models"  # replace with one of your models

# Load model as a PyFuncModel.
top3_3_loaded_model = mlflow.pyfunc.load_model(top3_3_logged_model)

best_model = mlflow.sklearn.load_model(top3_1_logged_model)
model_2 = mlflow.sklearn.load_model(top3_2_logged_model)
model_3 = mlflow.sklearn.load_model(top3_3_logged_model)

In [37]:
mlflow.set_experiment("Lab2-student_performance_output")
with mlflow.start_run():
    for model in [best_model, model_2, model_3]:
        params = model.get_params()
        for phase in ["train", "validation", "test"]:
            if phase == "train":
                acc = cross_val_score(model, X_redcued_train, y_train).mean()
            elif phase == "validation":
                acc = cross_val_score(model, X_redcued_val, y_val).mean()
            elif phase == "test":
                acc = cross_val_score(best_model, X_redcued_test, y_test).mean()

            mlflow.set_tag("Model", "df")
            mlflow.set_tag("Data", phase)
            mlflow.log_params(model.get_params())
            mlflow.log_metric("accuracy", acc)
            mlflow.sklearn.log_model(model, artifact_path="better_models")
            mlflow.end_run()

/opt/anaconda3/envs/mlops/lib/python3.12/site-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
2025/03/21 22:02:13 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
/opt/anaconda3/envs/mlops/lib/python3.12/site-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
2025/03/21 22:02:14 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
/opt/anaconda3/envs/mlops/lib/python3.12/site-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
2025/03/21 22:02:14 WARNING mlflow.models.